In [1]:
# from itertools import cycle, islice

# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# from scipy import signal
# from sklearn.model_selection import train_test_split
# # from tpot import TPOTClassifier
# from sklearn.externals import joblib
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import metrics
# import pandas
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.svm import SVR
# import numpy as np
# from sklearn.utils.multiclass import unique_labels
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import roc_curve, auc
# from pandas import read_csv
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier 
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# import scikitplot as skplt
# import matplotlib.pyplot as plt
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score


# Relax! This is only a game
A person under stress can affect their performance. Our interest lays in stress release video games that can reduce the abnormally high stress level. 

This project uses ECG signals to determine the stress level of a person before and after playing the stress release video games. To demonstrate the algorithm, different stress and non-stress ECG signals were used as input to the algorithm. After feature extraction, the features were fed into different statistical machine learning algorithms. The algorithms tested include Navie Bayes, Decision Tree and Gradient Boost. Among these algorithm Gradient Boost provided the highest accuracy. 

In [26]:
# Importing packages
import pandas as pd
import numpy as np
from numpy import set_printoptions

# Preparing dataset
from sklearn.preprocessing import MinMaxScaler
import random

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Cross Validation
from sklearn.model_selection import KFold

# Performance Measure
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import confusion_matrix
#from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
import statistics

# Plotting
import matplotlib.pyplot as plt

## Preparing the Dataset
Loading the feature csv file into arrays and normalized the features to range of [0,1]. There are 49 signals in total with 21 stress signals and 28 non-stress signals

In [17]:
# load the feature files into panda with X contains the features and Y contains the corresponding labels

filename = '~binhn/Downloads/stress_analysis.csv'
names = ['Mean HR','SDNN','RMSSD','NN50','PNN50','SD1','SD2','ApEn',
         'VLF(ms^2)','LF(ms^2)','HF(ms^2)','LF/HF ratio','TP',
         'VLF_lomb(ms^2)','LF_lomb(ms^2)','HF_lomb(ms^2)','LF/HF_lomb ratio','TP_lomb','Stress']
#dataframe = pd.read_csv(filename, names =names)
df = pd.read_csv(filename, names=names)
print(df.head(5))

array = df.values
X = np.array(df.iloc[:,0:18])
y = np.array(df.iloc[:,18])
y=y.reshape(-1,1)



   Mean HR      SDNN     RMSSD  NN50    PNN50       SD1       SD2     ApEn  \
0   84.760  0.053441  0.031241    21  0.20243  0.022112  0.072271  0.30596   
1   73.411  0.072690  0.083187   124  0.14045  0.058863  0.084277  0.69844   
2   76.055  0.160820  0.211560   148  0.25641  0.149790  0.171150  0.87521   
3   74.226  0.084564  0.102010    87  0.24691  0.072218  0.095325  0.75760   
4  123.880  0.088389  0.110670    60  0.16639  0.078318  0.097424  0.80389   

   VLF(ms^2)  LF(ms^2)  HF(ms^2)  LF/HF ratio       TP  VLF_lomb(ms^2)  \
0     12.406    3.9326    12.433      0.31630   30.541          400.21   
1    170.560   77.1350    48.860      1.57870  324.600         2078.80   
2    171.860  140.6900   235.750      0.59677  558.590         2899.80   
3    141.100   73.2180    66.160      1.10670  321.040         2123.20   
4    272.550   80.8040   166.750      0.48459  578.670          911.47   

   LF_lomb(ms^2)  HF_lomb(ms^2)  LF/HF_lomb ratio  TP_lomb  Stress  
0         846.71 

In [18]:
# Normalizing the features into [0,1]
scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
rescaledX = np.array(scaler.fit_transform(X)) # fit the trainning feature X into scaler

set_printoptions(precision=3) # how many decimal places.
print(rescaledX[0:5,:])



[[1.196e-01 1.906e-02 8.005e-03 3.125e-02 2.706e-01 8.005e-03 2.523e-02
  2.214e-03 1.925e-05 5.443e-06 3.416e-05 6.674e-02 1.642e-05 1.961e-03
  3.140e-03 3.528e-03 3.235e-01 3.756e-03]
 [7.900e-02 3.263e-02 3.440e-02 2.788e-01 8.984e-02 3.439e-02 3.352e-02
  5.225e-01 2.691e-04 1.105e-04 1.348e-04 5.928e-01 1.771e-04 1.167e-02
  1.245e-02 2.703e-02 1.629e-01 1.730e-02]
 [8.845e-02 9.475e-02 9.962e-02 3.365e-01 4.281e-01 9.967e-02 9.346e-02
  7.569e-01 2.712e-04 2.017e-04 6.511e-04 1.836e-01 3.050e-04 1.641e-02
  1.098e-02 1.472e-02 2.929e-01 1.667e-02]
 [8.191e-02 4.100e-02 4.396e-02 1.899e-01 4.004e-01 4.398e-02 4.114e-02
  6.010e-01 2.226e-04 1.048e-04 1.826e-04 3.961e-01 1.752e-04 1.192e-02
  9.496e-03 2.698e-02 1.120e-01 1.614e-02]
 [2.594e-01 4.369e-02 4.836e-02 1.250e-01 1.655e-01 4.836e-02 4.259e-02
  6.623e-01 4.302e-04 1.157e-04 4.604e-04 1.369e-01 3.159e-04 4.917e-03
  1.119e-02 4.622e-02 5.889e-02 2.541e-02]]


## Classification 
The classifiers tested in this project include Decision Tree, Each classifier uses 5-fold for validation.

In [27]:
# Classifiers
# Decision Tree
DT = DecisionTreeClassifier(random_state=0)
# Gradient Boost
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

# Support Vector Machine
SVM = SVC(random_state = 0, gamma='scale',probability = True)

# Linear Discrimant Analysis
LDA = LinearDiscriminantAnalysis()

In [52]:
# Classificaiton function with K-Fold cross-validation with user define iterations to get the standard deviation 

def classify(model, Iterations, num_folds, rescaledX, y):
    acc = []
    pre = []
    rec = []

    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        #results = cross_val_score(DT, rescaledX, y, cv = kfold)
        #print(results)

        for train_index, test_index in kfold.split(rescaledX):
            X_train, X_test = rescaledX[train_index], rescaledX[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # perform training and testing
            if model == SVC:
                model.fit(X_train,y_train)
            else:
                model.fit(X_train,y_train.ravel())
            #dtscores = DT.score(X_test,y_test)
            yPred = model.predict(X_test)


            # record performance
            acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
            pre = np.append(pre,metrics.precision_score(y_test, yPred, pos_label=1, average='macro'))
            rec = np.append(rec,metrics.recall_score(y_test, yPred, pos_label=1, average='macro'))

    return acc, pre, rec
    

    

In [38]:
Iterations = 100
num_folds = 5


### Linear Discrimant Analysis

In [53]:
print('Support Vector Machine')
acc, pre, rec = classify(LDA, Iterations, num_folds,  rescaledX, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Support Vector Machine


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anacond

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are col

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anacond

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anacond

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anacond

Accuracy (mean, std): 0.653 0.13984555848192287
Precision (mean, std): 0.6504650793650794 0.16554207674229324
Recall (mean, std): 0.6386603174603175 0.16554207674229324


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\binhn\Anacond

### Classification Using Decision Tree Classifier

In [49]:
print('Decision Tree')
acc, pre, rec = classify(DT, Iterations, num_folds,  rescaledX, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
            

Decision Tree


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy (mean, std): 0.7337333333333333 0.16341847163878956
Precision (mean, std): 0.7361456349206349 0.1743892198232936
Recall (mean, std): 0.7367400793650793 0.1743892198232936


### Support Vector Machine

In [50]:
print('Support Vector Machine')
acc, pre, rec = classify(SVM, Iterations, num_folds,  rescaledX, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Support Vector Machine


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\m

Accuracy (mean, std): 0.7545555555555555 0.1260606059795063
Precision (mean, std): 0.7882702380952381 0.14257041750371147
Recall (mean, std): 0.7342833333333333 0.14257041750371147


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\metrics\classific

### Classification using Gradient Boosting

In [19]:
print('Gradient Boosting')
acc, pre, rec = classify(GB, Iterations, num_folds,  rescaledX, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Gradient Boosting


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Accuracy (mean, std): 0.8568888888888889 0.13625932744128919
Precision (mean, std): 0.8683194444444444 0.13420292552762123
Recall (mean, std): 0.8512619047619048 0.13420292552762123


C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\binhn\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [110]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization DT')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix DT')

plt.show()



ValueError: Found input variables with inconsistent numbers of samples: [9, 10]

In [7]:
TP = matrix[1,1]
TN = matrix[0,0]
FP = matrix[0,1]
FN = matrix[1,0]


TP, TN, FP, FN

(4, 5, 1, 0)

In [8]:
#classification accuracy using TP and TN
print((TP + TN)/float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred))


0.9


NameError: name 'metrics' is not defined

In [ ]:
probs = models.predict_proba(X_test)

In [ ]:

# y_true = # ground truth labels
# y_probas = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc_curve(y_test, probs)
plt.show()

In [ ]:
false_positive_rate, recall, thresholds = roc_curve(y_test, probs[:,1])

In [ ]:
roc_auc = auc(false_positive_rate, recall)


In [ ]:
plt.title('ROC curve from DT')
plt.plot(false_positive_rate, recall, 'b', label='AUC + %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0,1],[0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()


In [ ]:

cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
y_pred_probs = models.predict_proba(X_test)[:,1]


In [ ]:
# precision recall curve
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(precision, recall, label = 'Precision-recall curve DT') # label = title of the graph
_= plt.xlabel('Precision')
_= plt.ylabel('Recall')
_= plt.title('Precision-recall curve DT')
_= plt.xlim([0.735, 1])
_= plt.ylim([0, 1.02])
_= plt.legend(loc='lower left')


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
min_samples_split=2, random_state=0)

In [ ]:
scores = cross_val_score(clf, rescaledX, rescaled_y, cv=5)
scores

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf2 = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf2, rescaledX, rescaled_y, cv=5)
scores

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
max_depth=1, random_state=0).fit(X_train, y_train)
clf3.score(X_test, y_test) 

In [ ]:
num_folds = 5
seed =7
kfold = KFold(n_splits = num_folds, random_state = seed)
results = cross_val_score(clf3, rescaledX, rescaled_y, cv = kfold)
print(results)
print("ACC: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100))
scores = clf3.score(X_test, y_test)
print(scores)

In [ ]:
y_pred = clf3.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.utils.multiclass import unique_labels

In [ ]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix

clf3.fit(X_train, y_train)
matrix = confusion_matrix(y_test, y_pred)
matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization GradientBoostingClassifier')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix GradientBoostingClassifier')

plt.show()



In [ ]:
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'], title='GradientBoostingClassifier')


In [ ]:
TP = matrix[1,1]
TN = matrix[0,0]
FP = matrix[0,1]
FN = matrix[1,0]

TP, TN, FP, FN

In [ ]:
#classification accuracy using TP and TN

print((TP + TN)/float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred))


In [ ]:
# classification error
classification_error = (FP + FN)/float(TP + TN + FP + FN)
print(classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred))


In [ ]:
# Sensitivity
sensitivity = TP/float(FN + TP)
print(sensitivity)
print(metrics.recall_score(y_test, y_pred))



In [ ]:
#specificity
specificity = TN/(TN + FP)
print(specificity)


In [ ]:
#false positive rate

false_positive_rate = FP / float(TN + FP)

print(false_positive_rate)
print(1 - specificity)


In [ ]:
# precision 
precision = TP / float(TP + FP)

print(precision)
print(metrics.precision_score(y_test, y_pred))


In [ ]:
#Cross Validation Classification Report
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

y_pred = clf3.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
probs = clf3.predict_proba(X_test)


In [ ]:
# y_true = # ground truth labels
# y_probas = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc_curve(y_test, probs)
plt.title('Gradient Boosting classifier ROC curves')
plt.show()

In [ ]:
false_positive_rate, recall, thresholds = roc_curve(y_test, probs[:,1])

In [ ]:
roc_auc = auc(false_positive_rate, recall)


In [ ]:
plt.title('ROC GradientBoostingClassifier')
plt.plot(false_positive_rate, recall, 'b', label = 'AUC + %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()


In [ ]:
y_pred_probs = clf3.predict_proba(X_test)[:,1]


In [ ]:
#precision recall curve
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(precision, recall,'r', label = 'Precision-recall curve GradientBoostingClassifier')
_= plt.xlabel('Precision')
_= plt.ylabel(' Recall')
_= plt.title('Precision-recall curve')
_= plt.xlim([0.735, 1])
_= plt.ylim([0, 1.02])
_= plt.legend(loc = 'lower left')


In [ ]:
#SVM classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC


model6 = SVC(random_state = 0, probability = True)
model6.fit(X_train, y_train)
num_folds = 5
seed =7
kfold = KFold(n_splits = num_folds, random_state = seed)
results = cross_val_score(model6, rescaledX, rescaled_y, cv = kfold)
print(results)
print("ACC: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100))
scores = model6.score(X_test, y_test)
print(scores)


In [ ]:
y_pred = model6.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
matrix

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from __future__ import print_function, division
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
sns.set(style='darkgrid', palette='colorblind', color_codes=True)

from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)

# from logitboost import LogitBoost

In [ ]:
# lboost = LogitBoost(DecisionTreeRegressor(max_depth = 3),
#                    n_estimators = 30, random_state=0)
# lboost.fit(X_train, y_train)

In [ ]:
# y_pred_train = lboost.predict(X_train)
# y_pred_test = lboost.predict(X_test)
# accuracy_train = accuracy_score(y_train, y_pred_train)
# accuracy_test = accuracy_score(y_test, y_pred_test)
# print('Trainning accuracy: %.4f' % accuracy_train)

# print('Test accuracy: %.4f' % accuracy_test)

In [ ]:
# report_train = classification_report(y_train, y_pred_train)
# report_test = classification_report(y_test, y_pred_test)
# print('Training\n%s' % report_train)
# print('Test\n%s' % report_test)

In [ ]:
# y_pred = lboost.predict(X_test)

In [ ]:
# matrix = confusion_matrix(y_test, y_pred)
# matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization GradientBoostingClassifier')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix GradientBoostingClassifier')

plt.show()

In [ ]:
# probs = lboost.predict_proba(X_test)

In [ ]:
# false_positive_rate, recall, thresholds = roc_curve(y_test, probs[:,1])

In [ ]:
# roc_auc = auc(false_positive_rate, recall)

In [ ]:
# plt.title('Receiver Operating Characteristic LogitBoost')
# plt.plot(false_positive_rate, recall, 'b', label='AUC + %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0,1],[0,1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.ylabel("Recall")
# plt.xlabel('Fall-out')
# plt.show()

In [ ]:
# y_pred_probs = lboost.predict_proba(X_test)[:,1]

In [ ]:
# precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
# plt.plot(precision, recall, label = 'Precision-recall curve LogitBoost') 
# _= plt.xlabel('Precision')
# _= plt.ylabel('Recall')
# _= plt.title('Precision-recall curve')
# _= plt.xlim([0.735, 1])
# _= plt.ylim([0, 1.02])
# _= plt.legend(loc='lower left')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
num_folds = 5
seed =7
kfold = KFold(n_splits = num_folds, random_state = seed)
results = cross_val_score(knn, rescaledX, rescaled_y, cv = kfold)
print(results)
print("ACC: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100))
scores = knn.score(X_test, y_test)
print(scores)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization KNN')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix KNN')

plt.show()

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
probs = knn.predict_proba(X_test)

In [ ]:
false_positive_rate, recall, threshold = roc_curve(y_test, probs[:,1])

In [ ]:
# y_true = # ground truth labels
# y_probas = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc_curve(y_test, probs)
plt.title('KNN ROC Curves')
plt.show()

In [ ]:
roc_auc = auc(false_positive_rate, recall)

In [ ]:
plt.title('ROC curve KNN')
plt.plot(false_positive_rate, recall, 'b', label = 'AUC + %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()

In [ ]:
y_pred_probs = knn.predict_proba(X_test)[:,1]

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(precision, recall, label = 'Precision-recall curve KNN')
_= plt.xlabel('Precision')
_= plt.ylabel('Recall')
_= plt.title('Precision-recall curve')
_= plt.xlim([0.735, 1])
_= plt.ylim([0, 1.02])
_= plt.legend(loc='lower left')


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()

In [ ]:
LDA = LinearDiscriminantAnalysis()

In [ ]:
LDA.fit(X_train, y_train)

In [ ]:
y_pred = LDA.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
num_folds = 5
seed =7
kfold = KFold(n_splits = num_folds, random_state = seed)
results = cross_val_score(LDA, rescaledX, rescaled_y, cv = kfold)
print(results)
print("ACC: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100))
scores = LDA.score(X_test, y_test)
print(scores)

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization LDA')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix LDA')

plt.show()


In [ ]:
probs = LDA.predict_proba(X_test)

In [ ]:
false_positive_rate, recall, threshold = roc_curve(y_test, probs[:,1])

In [ ]:
roc_auc = auc(false_positive_rate, recall)

In [ ]:
# y_true = # ground truth labels
# y_probas = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc_curve(y_test, probs)
plt.title('ROC Curves LDA')
plt.show()

In [ ]:
plt.title('ROC cuve LDA')
plt.plot(false_positive_rate, recall, 'b', label = 'AUC + %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()

In [ ]:
y_pred_probs = LDA.predict_proba(X_test)[:,1]

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(precision, recall, label = 'Precision-recall curve LDA')
_= plt.xlabel('Precision')
_= plt.ylabel('Recall')
_= plt.title('Precision-recall curve LDA')
_= plt.xlim([0.735, 1])
_= plt.ylim([0, 1.02])
_= plt.legend(loc = 'lower left')


In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
NB = GaussianNB()
NB.fit(X_train, y_train)
NB.score(X_test,y_test)

In [ ]:
num_folds = 5
seed =7
kfold = KFold(n_splits = num_folds, random_state = seed)
results = cross_val_score(NB, rescaledX, rescaled_y, cv = kfold)
print(results)
print("ACC: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100))
scores = NB.score(X_test, y_test)
print(scores)

In [ ]:
y_pred = NB.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print(cm)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization GaussianNB')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix GaussianNB')

plt.show()


In [ ]:
probs = NB.predict_proba(X_test)

In [ ]:
# y_true = # ground truth labels
# y_probas = # predicted probabilities generated by sklearn classifier
skplt.metrics.plot_roc_curve(y_test, probs)
plt.title('ROC curves GaussianNB')
plt.show()

In [ ]:
false_positive_rate, recall, threshold = roc_curve(y_test, probs[:,1])

In [ ]:
roc_auc = auc(false_positive_rate, recall)

In [ ]:
plt.title('ROC cuve GaussianNB')
plt.plot(false_positive_rate, recall, 'b', label = 'AUC + %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()

In [ ]:
cr = classification_report(y_pred, y_test)
print(cr)

In [ ]:
model = NB

In [ ]:
y_pred_probs = model.predict_proba(X_test)[:,1]

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(precision, recall, label = 'Precision-recall curve GaussianNB')
_= plt.xlabel('Precision')
_= plt.ylabel('Recall')
_= plt.title('Precision-recall curve GaussianNB')
_= plt.xlim([0.735, 1])
_= plt.ylim([0, 1.02])
_= plt.legend(loc = 'lower left')

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)

In [ ]:
y_pred = svm.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed', 'Stressed'],
                      title='Confusion matrix, without normalization SVM')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=['Relaxed','Stressed'], normalize=True,
                      title='Normalized confusion matrix SVM')

plt.show()


In [ ]:
probs = svm.predict_proba(X_test)